In [2]:
%matplotlib inline

# TF-IDF + SVM 을 이용한 News Sentiment 분석
## Process
1. 미리 Preprocessing된 데이터를 읽어
2. 데이터를 TF-IDF로 임베딩
3. TRAIN-TEST 데이터 분리
4. SVM으로 TRAIN 학습
5. TEST 검증
6. 최적의 k값 찾기

## Reference
- [Word Embedding Explained, a comparison and code tutorial](https://medium.com/@dcameronsteinke/tf-idf-vs-word-embedding-a-comparison-and-code-tutorial-5ba341379ab0)
- [Scikit-Learn의 문서 전처리 기능](https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/)
- [서포트 벡터 머신](https://datascienceschool.net/view-notebook/6c6d450cb2ee49558856fd924b326e00/)

In [19]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [4]:
# 데이터 불러옴
data = pd.read_json("../data/news_reactions_recent_preprocessed.json")
data.head()

,title_token,content_token,reaction_category
0,"[전두환, 프로젝트, 연희동, 집, 가구, 세트, 막대, 세금, 구입, 네이버, 뉴스]","[대통령, 자리, 물러난, 전두환, 별도, 전직, 대통령, 사무실, 내지, 않았으면...",1
1,"[식약처, 존슨, 앤드, 존슨, 제품, 빼고, 암, 위험, 인공, 유방, 퇴, 출,...","[식품의약품안전처, 암, 발병, 사례, 보고, 된, 인공, 유방, 보, 형, 물이,...",1
10,"[늘어나는, 주택연금, 가입자, 집, 한, 채, 있다면, 노후, 걱정, 마세요, 네...","[주택연금, 올, 들어서만, 명, 가입, 정부, 주택연금, 가입, 기준, 완화, 예...",2
100,"[비서, 성폭행, 안희정, 심, 달랐던, 판단, 오늘, 대법, 선고, 네이버, 뉴스]","[앵커, 심, 무죄, 나왔지만, 심, 실형, 선고, 받고, 법정구속, 돼, 있습니다...",1
1000,"[인천, 경기, 북부, 호우, 주의보, 내일, 최고, 더, 온다, 네이버, 뉴스]","[아침, 인천, 경기, 북부, 파주, 고양, 연천, 김포, 안산, 호우, 주의보, ...",2


## Vectorization
Scikit-Learn의 TfidVectorizer를 이용해서 TF-IDF 임베딩을 수행함

In [45]:
MAX_FEATURE = 2048
title_vectorizer = TfidfVectorizer (max_features=MAX_FEATURE)
content_vectorizer = TfidfVectorizer (max_features=MAX_FEATURE)

In [46]:
%%time 
title_corpus = data.title_token.map(lambda x: " ".join(x))
title_vectorizer.fit(title_corpus)

CPU times: user 62.7 ms, sys: 19.8 ms, total: 82.4 ms
Wall time: 88.9 ms


In [47]:
%%time
content_corpus = data.content_token.map(lambda x: " ".join(x))
content_vectorizer.fit(content_corpus)

CPU times: user 892 ms, sys: 182 ms, total: 1.07 s
Wall time: 1.16 s


위에서 생성한 Vectorizer를 이용해서 문장 내의 토큰들을 TF-IDF 벡터로 변환한다.
그러면 한 뉴스 문서는 TF-IDF 벡터들의 목록이므로 2차원 배열이 됨.

In [48]:
%%time 
TITLE_MAX_LEN = 32
CONTENT_MAX_LEN = 512
title_X = data.title_token.map(lambda x: title_vectorizer.transform(x if len(x) <= TITLE_MAX_LEN else x[:TITLE_MAX_LEN]))
content_X = data.title_token.map(lambda x: content_vectorizer.transform(x if len(x) <= CONTENT_MAX_LEN else x[:CONTENT_MAX_LEN]))

CPU times: user 2.28 s, sys: 30.4 ms, total: 2.31 s
Wall time: 2.42 s


In [ ]:
print(title_vectorizer.vocabulary_)

In [50]:
# sparse matrix로 결과가 나오는 모습이다
print(title_X[0].shape)
print(title_X[0].toarray())
print(content_X[0].shape)
print(content_X[0].toarray())

(11, 2048)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(11, 2048)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [51]:
y = data.reaction_category
print(len(y), y[:5])

2302 0       1
1       1
10      2
100     1
1000    2
Name: reaction_category, dtype: int64


In [52]:
%%time 
# add padding and merge title and content
def add_padding(x, min_size):
    if len(x) <= min_size:
        pad = np.zeros((min_size - len(x), x.shape[1]), dtype=float)
        return np.concatenate([x, pad], axis=0)
    return x

# a = title_X[0]
# r = add_padding(a.toarray(), TITLE_MAX_LEN)
# print(r)
# print(r.shape)
title_X = title_X.map(lambda x: add_padding(x.toarray(), TITLE_MAX_LEN))
content_X = content_X.map(lambda x: add_padding(x.toarray(), CONTENT_MAX_LEN))

CPU times: user 10.3 s, sys: 12 s, total: 22.3 s
Wall time: 23.5 s


In [53]:
title_X = np.asarray(title_X)
content_X = np.asarray(content_X)

In [54]:
# 뉴스 제목이랑 뉴스 내용이랑 합친다.
X = [np.concatenate([title_X[i], content_X[i]], axis=0) for i in range(len(title_X))]

# 메모리없어서 자꾸 터짐 ㅠ
del title_X
del content_X

In [55]:
# Shape check
print(len(X))
print(X[0], X[0].shape)

2302
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (544, 2048)


## 학습하기
sklearn.svm.SVM을 이용한다.


In [56]:
# Scikit Learn은 학습데이터가 2d array여야해서 강제로 변경
nsamples = len(X)
nx, ny = X[0].shape
X = np.reshape(X, (nsamples, nx*ny))

In [57]:
# 학습, 실험 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

이제 SVC를 이용해서 학습한다.

In [58]:
%%time 

model = LinearSVC(verbose=1, max_iter=1000)
model.fit(X_train, y_train)

[LibLinear]CPU times: user 9.32 s, sys: 17.2 s, total: 26.5 s
Wall time: 30.4 s


score() 함수를 이용해서 평가한다.

In [59]:
model.score(X_test, y_test)

0.544468546637744

Test 결과의 몇개 예시를 만들어본다

In [60]:
test_i = [0, 10, 50]
sentiment = ["긍정", "부정", "중립"]
for i in test_i:
    x = X_test[i]
    y = model.predict([x])[0]
    print(data.loc[i])
    print("Predict:", sentiment[y])
    print("Answer:", sentiment[y_test.values[i]])
    print()

title_token           [전두환, 프로젝트, 연희동, 집, 가구, 세트, 막대, 세금, 구입, 네이버, 뉴스]
content_token        [대통령, 자리, 물러난, 전두환, 별도, 전직, 대통령, 사무실, 내지, 않았으면...
reaction_category                                                    1
Name: 0, dtype: object
Predict: 부정
Answer: 부정

title_token          [늘어나는, 주택연금, 가입자, 집, 한, 채, 있다면, 노후, 걱정, 마세요, 네...
content_token        [주택연금, 올, 들어서만, 명, 가입, 정부, 주택연금, 가입, 기준, 완화, 예...
reaction_category                                                    2
Name: 10, dtype: object
Predict: 중립
Answer: 중립

title_token                    [검찰, 없는, 컬러, 표창장, 박지원, 입수, 경로, 네이버, 뉴스]
content_token        [꿈, 담는, 캔버스, 채널, A, CHANNEL, A, 무단, 재, 및, 재, 배...
reaction_category                                                    1
Name: 50, dtype: object
Predict: 긍정
Answer: 부정



In [65]:
# 결과 모델을 pickle 파일로 저장합니다.
import pickle
from datetime import datetime
t = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
pickle.dump(model, open(f"../model/tfidf_svm_{t}.pickle", "wb"))